In [ ]:
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [15]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [17]:
embeddings.embed_query("hello AI")

[-0.033388182520866394,
 0.03453972190618515,
 0.059474531561136246,
 0.05928609147667885,
 -0.0635354220867157,
 -0.06819586455821991,
 0.08823323994874954,
 0.03444080427289009,
 -0.03278516232967377,
 -0.015814989805221558,
 0.02098178118467331,
 -0.01834029331803322,
 -0.03983215242624283,
 -0.0804707482457161,
 -0.014469144865870476,
 0.0332648828625679,
 0.014259284362196922,
 -0.03404996916651726,
 -0.142915740609169,
 -0.023083344101905823,
 -0.021380102261900902,
 0.002633501309901476,
 -0.047292742878198624,
 -0.010752756148576736,
 -0.06866798549890518,
 0.031125057488679886,
 0.0759458914399147,
 0.0011283254716545343,
 0.011631987057626247,
 -0.03603919595479965,
 0.04483763128519058,
 0.018390750512480736,
 0.12672801315784454,
 -0.0013597895158454776,
 0.008206663653254509,
 0.06909968703985214,
 -0.08076353371143341,
 -0.05841314047574997,
 0.053754497319459915,
 0.026227595284581184,
 -0.006828607991337776,
 -0.056358352303504944,
 0.0032930178567767143,
 -0.0725017860

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [19]:
my_query="Narendra modi is prime minister of india?"

In [20]:
document_embedding=embeddings.embed_documents(documents)

In [21]:
document_embedding

[[0.11998696625232697,
  -0.021302605047822,
  -0.04288087412714958,
  0.06645582616329193,
  -0.0643523558974266,
  -0.04424864798784256,
  0.022408470511436462,
  -0.049873027950525284,
  -0.023437663912773132,
  -0.03397207707166672,
  -0.0140480762347579,
  -0.06065931171178818,
  -0.003906761296093464,
  -0.017782077193260193,
  -0.047971006482839584,
  -0.0666816309094429,
  0.004103219136595726,
  -0.013092794455587864,
  0.0443977415561676,
  0.022350674495100975,
  0.009459568187594414,
  -0.02056453935801983,
  -0.00033560290466994047,
  -0.005685777403414249,
  0.05558697134256363,
  0.025123219937086105,
  -0.0028171155136078596,
  0.008759002201259136,
  0.003255249932408333,
  -0.015963444486260414,
  0.014263702556490898,
  -0.11220847815275192,
  0.0896855890750885,
  -0.031083744019269943,
  -0.02422383241355419,
  0.006152077112346888,
  0.08058709651231766,
  0.018250005319714546,
  0.05568312108516693,
  0.016702702268958092,
  0.01589597389101982,
  0.0003410273930

In [22]:
query_embedding=embeddings.embed_query(my_query)

In [23]:
len(query_embedding)

384

In [24]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756668, 0.3432456 , 0.81413237]])

In [25]:
from sklearn.metrics.pairwise import euclidean_distances

In [26]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848283, 1.14608416, 0.60970098]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [27]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [28]:
index=faiss.IndexFlatL2(384)


In [29]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023B9A4DD1A0> >

In [30]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [31]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['34f012dc-3b17-4dc2-b42d-d4bc2e0ed085',
 'c416dcef-6fca-4098-b514-df9da3bf378d',
 '52829be6-dc21-451c-b370-563b29f8f9df']

In [32]:
vector_store.index_to_docstore_id

{0: '34f012dc-3b17-4dc2-b42d-d4bc2e0ed085',
 1: 'c416dcef-6fca-4098-b514-df9da3bf378d',
 2: '52829be6-dc21-451c-b370-563b29f8f9df'}

In [33]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [34]:
results


[Document(id='c416dcef-6fca-4098-b514-df9da3bf378d', metadata={}, page_content='AI is powerful'),
 Document(id='34f012dc-3b17-4dc2-b42d-d4bc2e0ed085', metadata={}, page_content='AI is future'),
 Document(id='52829be6-dc21-451c-b370-563b29f8f9df', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [47]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [48]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [49]:
vector_store.add_documents(documents=documents)

['4c433a1f-a87e-4859-a69d-b1b5aff1b234',
 '7e305a4c-3648-4a03-8485-e205004bbf19',
 '3cf6d552-9f43-4dd2-b713-2c60e4d80751',
 '1a97a065-5424-4b7b-99a5-146faafa39de',
 'ab3e5493-4aca-42e8-b369-beee76b7af4a',
 '31171995-2d3e-43ca-af5e-8342f9c6bd9e',
 'b8d0307e-c893-4caa-b391-43173c57e1b5',
 '4e37af39-fefc-4344-bb96-6a9cd39a6a38',
 '9297f916-6253-4ed7-8e8d-fe728e790260',
 'db2fe3da-3cd2-4def-b542-c7bb4d66c307']

In [39]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='6869d6ef-e9e2-4c09-b6f5-65da50d4771b', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7984774-7f52-4b0c-a4e6-3889ec41c832', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [40]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='6869d6ef-e9e2-4c09-b6f5-65da50d4771b', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7984774-7f52-4b0c-a4e6-3889ec41c832', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='daccd4a0-edee-41e9-bb75-fc7f0ee7f6bf', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='309d847b-1d76-48ec-8ca6-3360fecd5a63', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [50]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [42]:
result[0].metadata

{'source': 'news'}

In [51]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [44]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [45]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='6869d6ef-e9e2-4c09-b6f5-65da50d4771b', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7984774-7f52-4b0c-a4e6-3889ec41c832', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='daccd4a0-edee-41e9-bb75-fc7f0ee7f6bf', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

inmemory(server)
ondisk(server)
cloud(yet to discuss)

In [54]:
vector_store.save_local("today's class faiss index")

In [57]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)
new_vector_store

In [58]:
new_vector_store.similarity_search("langchain")

[Document(id='3cf6d552-9f43-4dd2-b713-2c60e4d80751', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4e37af39-fefc-4344-bb96-6a9cd39a6a38', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='ab3e5493-4aca-42e8-b369-beee76b7af4a', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='b8d0307e-c893-4caa-b391-43173c57e1b5', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [12]:
import os
print("Current directory:", os.getcwd())


Current directory: c:\Users\Nahid\OneDrive - Calmi2\Desktop\learning-portfolio\Agentic AI\Agentic-AI-Projects\3-Data Transformer\2.4-VectorDatabase\FAISS


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Since llama2.pdf is in the same directory, just use the filename
FILE_PATH =r"C:\Users\Nahid\OneDrive - Calmi2\Desktop\learning-portfolio\Agentic AI\Agentic-AI-Projects\3-Data Transformer\2.4-VectorDatabase\llama2.pdf"

loader = PyPDFLoader(FILE_PATH)
documents = loader.load()

# Preview the first chunk
print(documents[0].page_content)


Llama 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗ Louis Martin† Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov

In [15]:
len(loader.load())

77

In [16]:
pages=loader.load()
pages

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\Nahid\\OneDrive - Calmi2\\Desktop\\learning-portfolio\\Agentic AI\\Agentic-AI-Projects\\3-Data Transformer\\2.4-VectorDatabase\\llama2.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini R

In [18]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)
pages

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\Nahid\\OneDrive - Calmi2\\Desktop\\learning-portfolio\\Agentic AI\\Agentic-AI-Projects\\3-Data Transformer\\2.4-VectorDatabase\\llama2.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini R

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [26]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [27]:
split_docs = splitter.split_documents(pages)

In [28]:
len(split_docs)

615

In [29]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [30]:
vector_store.add_documents(documents=split_docs)

['a74a05cd-5626-4feb-9e9f-bb94f5238f77',
 'c8f59b12-067e-4c90-b4a2-a8b0b4363cc6',
 'dad04540-9744-4106-9a42-529241f8849c',
 'b1ce43db-1ebf-4357-84e0-d7e5fc28a11c',
 '1ec5041b-6ec7-4ec0-9cbd-48413d9aa15e',
 'de620dae-508d-4666-930e-c627fd0b5ee3',
 'b030a5bd-2d8f-4402-8dea-d4b7b88a0137',
 'd8b54090-a9ce-4105-a7d5-c8cbdff66d32',
 '9ba60e03-0f93-4cb2-824c-6005f2d275c3',
 '03c4296a-ff0f-44c7-b743-a00027ba3eaf',
 '1ab91611-bdb0-462d-ab98-c53610e0f616',
 '120613a5-197d-4a4a-9b04-ca36960fbcac',
 '242e2716-c2ba-4cab-a726-5fa67b381aff',
 '9f791206-d812-48b6-83ad-06090c8b98bf',
 '5819a0a1-b7a1-4c6e-a48b-555b75c25e7e',
 '35d25e76-5236-4d51-abda-68ea66cefadb',
 '11cd7510-5646-44bd-8456-b6b03b61745c',
 '1559fbfc-b3ab-4c58-ac33-61726ac7c770',
 'db941825-36a1-443c-ac9e-0f98223d6d11',
 '725bc366-26e3-40fd-869e-15e77cc28d1f',
 '11a37dc9-e6e7-4966-99e1-8dc2005fadaa',
 '525e2aad-66e3-449f-b563-621e195f0820',
 '3ce17f8a-2aba-49b5-8173-380d5b0aeff4',
 '438be3c6-bb6b-43c8-81dc-7b816b9e003e',
 '014764c8-2ef7-

In [31]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [32]:
retriever.invoke("what is llama model?")

[Document(id='014764c8-2ef7-4a27-aa7a-9a5dde3a2285', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\Nahid\\OneDrive - Calmi2\\Desktop\\learning-portfolio\\Agentic AI\\Agentic-AI-Projects\\3-Data Transformer\\2.4-VectorDatabase\\llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='254c963c-9401-4c3c-9186-e78e029126a3', metadata={'producer': 'pdfTeX-1.40.25', 

In [33]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEmbeddings
import faiss
import os

# Step 1: Define PDF file path (assumes llama2.pdf is in the current working directory)


# Step 2: Load PDF pages
loader = PyPDFLoader(FILE_PATH)
pages = loader.load()

# Step 3: Split pages into overlapping chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(pages)

# Step 4: Initialize Hugging Face embeddings (384-dimensional vectors)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 5: Create FAISS index (IP = inner product, for cosine similarity with normalized vectors)
index = faiss.IndexFlatIP(384)

# Step 6: Create LangChain FAISS vector store
pdf_vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# Step 7: Add documents to the vector store
pdf_vector_store.add_documents(split_docs)

# Step 8: Create a retriever with top-k = 10 most similar chunks
retriever = pdf_vector_store.as_retriever(
    search_kwargs={"k": 10}  # number of chunks to return
)

# Step 9: Invoke the retriever with a user query
results = retriever.invoke("what is llama model?")

# Step 10: Display results
print("\n🔍 Top Results:")
for i, doc in enumerate(results, 1):
    print(f"\n--- Chunk {i} ---")
    print(doc.page_content)



🔍 Top Results:

--- Chunk 1 ---
work (Section 6), and conclusions (Section 7).
‡https://ai.meta.com/resources/models-and-libraries/llama/
§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.
¶https://ai.meta.com/llama
‖https://github.com/facebookresearch/llama
4

--- Chunk 2 ---
10.18653/v1/2022.gebnlp-1.13. URLhttps://aclanthology.org/2022.gebnlp-1.13.
AlonTalmor,JonathanHerzig,NicholasLourie,andJonathanBerant.Commonsenseqa: Aquestionanswering
challenge targeting commonsense knowledge.arXiv preprint arXiv:1811.00937, 2018.
Rohan Taori, Ishaan Gulrajani, Tianyi Zhang, Yann Dubois, Xuechen Li, Carlos Guestrin, Percy Liang, and
Tatsunori B. Hashimoto. Stanford alpaca: An instruction-following llama model.https://github.com/
tatsu-lab/stanford_alpaca, 2023.

--- Chunk 3 ---
models-and-libraries/llama-downloads/
Where to send com-
ments
Instructions on how to provide feedback or comments on the model can be
found in the model README, or by opening 

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [51]:
# Import necessary modules
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pprint

# Initialize the OpenAI model
model = ChatOpenAI(model="gpt-4o")

# Load the RAG prompt template from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Display the loaded prompt structure
pprint.pprint(prompt.messages)

# Define a helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG pipeline (make sure 'retriever' is defined elsewhere)
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

# Run the RAG pipeline with a sample question
response = rag_chain.invoke("what is llama model?")
print(response)





[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
The LLaMA (Large Language Model Meta AI) model, developed by Meta, is a series of large language models intended for both research and commercial use, performing various natural language generation tasks. LLaMA 2, a subsequent version, features enhanced performance and safety through methods like supervised fine-tuning and reinforcement learning with human feedback. These models are available in various sizes, with the LLaMA 2-Chat models optimized for assistant-like chat functions.
